In [ ]:
import torch
import torch.nn as nn
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
import pdb
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install --upgrade datasets
dataset = load_dataset("cnn_dailymail","3.0.0")

In [ ]:
!pip install git
!git clone https://github.com/osamakhaled123/Basic-Transformer-Model
!cd '/content/Basic-Transformer-Model'
%cd Basic-Transformer-Model/

In [ ]:
import Processing_Summarizing_Datasets_From_Scratch as pre

In [ ]:
articles = np.array(dataset['train'][:14000]['article'])
summaries = np.array(dataset['train'][:14000]['highlights'])
texts = {'articles':articles,
         'summaries':summaries}
max_input_len = 1000
max_target_len = 85

In [ ]:
text_train, train_data, train_target, vocab = pre.preprocessing(texts, max_input_len, max_target_len, {})

In [ ]:
text_train.shape, train_data.shape, train_target.shape, len(vocab)

In [ ]:
train_set, val_set, test_set = pre.splitting_and_batching(input_data=train_data, target_data=train_target,
                                                          split_frac=0.95, batch_size=32) # Reduced batch size

In [ ]:
import Transformer_model

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = Transformer_model.Transformer(
#     vocab=vocab,
#     max_input_length=max_input_len,
#     max_target_length=max_target_len,
#     emb_dim=256,
#     dff=1024,
#     num_heads=4,
#     num_encoder_blocks=4,
#     num_decoder_blocks=4,
#     dropout=0.5
# ).to(device)

In [ ]:
# train_losses , val_losses = Transformer_model.train(model, train_set, val_set, 5, 0.005, device)

In [ ]:
# saved_dict ={'model':model,
#             'train loss':train_losses,
#             'val losses':val_losses}

In [ ]:
# torch.save(saved_dict, 'Transformer_Model.pt')

In [ ]:
%cd /kaggle/working/

In [ ]:
!pip install -q gdown
!gdown --id 1cxLyCAa4XH5xgaqV8CwTD_un3FEF1hGN --output Transformer_Model.pt


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
state_dict = torch.load('Transformer_Model.pt', map_location=device, weights_only=False)

In [ ]:
model = state_dict['model']
train_losses = state_dict['train loss']
val_losses = state_dict['val losses']

In [ ]:
sentence = """Harry Potter star Daniel Radcliffe gains access to a reported £20 million 
($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell 
on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the 
disappointment of gossip columnists around the world, the young actor says he has no plans to 
fritter his cash away on fast cars, drink and celebrity parties."""

In [ ]:
generated = Transformer_model.summarize(model, sentence, 3, device, 10)

In [ ]:
generated

In [ ]:
sns.set(style='whitegrid')

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed = []
    for p in points:
        if smoothed:
            smoothed.append(smoothed[-1] * factor + p * (1 - factor))
        else:
            smoothed.append(p)
    return smoothed

In [ ]:
def plot_losses(train_losses, val_losses):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, label='Training Loss', color='royalblue', linewidth=2.5, marker='o')
    plt.plot(epochs, val_losses, label='Validation Loss', color='tomato', linewidth=2.5, marker='s')
    
    # Highlight min val loss
    min_val_idx = val_losses.index(min(val_losses))
    plt.scatter(min_val_idx + 1, val_losses[min_val_idx], color='green', s=100, label='Best Validation Loss')

    plt.title('Training vs Validation Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=13)
    plt.ylabel('Loss', fontsize=13)
    plt.xticks(epochs)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_losses(smooth_curve(train_losses), smooth_curve(val_losses))